In [8]:
import model_feedinput_pipeline
from model_feedinput_pipeline import CODE_ENV

In [11]:
select_input_stepsize= 3000

#####################################################################################
#***************IMP: Update coding environment********************
#####################################################################################
code_env = CODE_ENV.WIN

dataset_paths = model_feedinput_pipeline.get_dataset_paths(code_env)


Number of files in 1st Dataset: 2156 first file= g:/My Drive/github/bearing-vibration-anomaly-detection/models/capstone-data/01_PHM-Bearing/1st_test/2003.10.22.12.06.24
Number of files in 2nd Dataset: 984 first file= g:/My Drive/github/bearing-vibration-anomaly-detection/models/capstone-data/01_PHM-Bearing/2nd_test/2004.02.12.10.32.39
Number of files in 3rd Dataset: 6324 first file= g:/My Drive/github/bearing-vibration-anomaly-detection/models/capstone-data/01_PHM-Bearing/3rd_test/2004.03.04.09.27.46
